<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/ai_serving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="blue">**Serving (Inference)**